In [14]:
import numpy as np
import glob
import cv2
import os
import pandas as pd
import json
import requests
import base64
from PIL import Image
from io import BytesIO
# List of directories present in the dataset

print(os.listdir('Plant/'))
'''Loading x i.e. images and y i.e. labels from the dataset'''

images = []
labels = []
for dir_path in glob.glob("Plant/*"):
    label = dir_path.split('\\')[-1]
    print(label)
    for img_path in glob.glob(os.path.join(dir_path, '*.jpg')):
        # print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (256, 256))

        images.append(img)
        labels.append(label)

images = np.array(images)
labels = np.array(labels)
print(len(images))
print(len(labels))
# LabelEncodeing Labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(labels)
print(y)
def feature_extractor(dataset):
    img_dataset = pd.DataFrame()
    for image in range(dataset.shape[0]):
        df = pd.DataFrame()

        input_img = dataset[image, :, :, :] # image , size, size, channels
        img = input_img
        # feature 1 - pixel values
        pixel_values = img.reshape(-1)
        df['pixel_value'] = pixel_values

        # feature 2 - gabor filter
        num = 1
        kernels = []
        for theta in range(2):
            theta = theta / 4. * np.pi
            for sigma in (1, 3):
                lamda = np.pi/4
                gamma = 0.5
                gabor_label = 'Gabor' + str(num)
                ksize=9 #kernel size
                kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma)
                kernels.append(kernel)
                fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel) # type: ignore
                filtered_image = fimg.reshape(-1)
                df[gabor_label] = filtered_image
                num+=1

        img_dataset = pd.concat([img_dataset, df], ignore_index=True)
    return img_dataset

image_features = feature_extractor(images)
print(image_features.shape)
image_features = np.expand_dims(image_features, axis=0)
x = np.reshape(image_features, (images.shape[0], -1))
# Train-Test Split


['Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']
Potato___Early_blight
Potato___healthy
Potato___Late_blight
1544
1544
[0 0 0 ... 1 1 1]


KeyboardInterrupt: 

In [2]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=42)
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)
lr_acc = accuracy_score(y_test, lr_pred)
print("Accuracy from Logistic Regression : " + str(lr_acc))

Accuracy from Logistic Regression : 0.8770226537216829


In [4]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(x_train, y_train)
rfc_pred = rfc.predict(x_test)
rfc_acc = accuracy_score(y_test, rfc_pred)
print("Accuracy from Random Forest Classifier : " + str(rfc_acc))

Accuracy from Random Forest Classifier : 0.8576051779935275


In [5]:
from sklearn.linear_model import Perceptron
p = Perceptron()
p.fit(x_train, y_train)
p_pred= p.predict(x_test)
p_acc = accuracy_score(y_test, p_pred)
print("Accuracy from Perceptron : " + str(p_acc))

Accuracy from Perceptron : 0.8964401294498382


In [6]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train, y_train)
svc_pred = svc.predict(x_test)
svc_acc = accuracy_score(y_test, svc_pred)
print("Accuracy from Support Vector Classifier : " + str(svc_acc))

Accuracy from Support Vector Classifier : 0.8802588996763754


In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test)
knn_acc = accuracy_score(y_test, knn_pred)
print("Accuracy from K Nearest Neighbours Classifier : " + str(knn_acc))

Accuracy from K Nearest Neighbours Classifier : 0.45307443365695793


In [8]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb_pred = gnb.predict(x_test)
gnb_acc = accuracy_score(y_test, gnb_pred)
print("Accuracy from Gaussian Naive Bayes : " + str(gnb_acc))

Accuracy from Gaussian Naive Bayes : 0.8414239482200647


In [10]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
kf = KFold(n_splits=5)
score = cross_val_score(logReg, x, y, cv=kf)
print(score)
print(score.mean())

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sujja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sujja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sujja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0